In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
import warnings

warnings.filterwarnings('ignore')

In [3]:
external_df = pd.read_csv("C:/Users/aruna/OneDrive/Documents/daigt_external_dataset.csv", sep=',')

external_df = external_df.rename(columns={'generated': 'label'})
external_df = external_df[["source_text"]]
external_df.columns = ["text"]
external_df['text'] = external_df['text'].str.replace('\n', '')
external_df["label"] = 1
external_df

,text,label
0,When considering the pros and cons of attendin...,1
1,One of the primary arguments for implementing ...,1
2,Before making any decisions about getting invo...,1
3,"Ralph Waldo Emerson once said, ""Go confidently...",1
4,Human character traits are shaped by a wide ra...,1
...,...,...
2416,Becoming a surgeon requires a great deal of de...,1
2417,Schools should offer an after school homework ...,1
2418,It’s human nature to be afraid to make mistake...,1
2419,One of the main debates of 2020 for many stude...,1


In [4]:
train = pd.read_csv("C:/Users/aruna/Downloads/train_essays_RDizzl3_seven_v1.csv")
df=pd.concat([train,external_df])

df.reset_index(drop=True, inplace=True)
df


,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
18287,Becoming a surgeon requires a great deal of de...,1
18288,Schools should offer an after school homework ...,1
18289,It’s human nature to be afraid to make mistake...,1
18290,One of the main debates of 2020 for many stude...,1


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3),sublinear_tf=True)
X = vectorizer.fit_transform(df['text'])
Y=df['label']

In [6]:
lr_model = LogisticRegression(solver="liblinear")

In [7]:
sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
ensemble = VotingClassifier(estimators=[('lr', lr_model),('sgd', sgd_model)], voting='soft')
ensemble.fit(X_train, y_train)


In [ ]:
import joblib
joblib.dump(vectorizer, 'vectorizer.joblib')

In [ ]:
pred_probabilities = ensemble.predict_proba(X_test)
pred=ensemble.predict(X_test)
pred_probabilities


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


conf_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])

plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Assuming y_test is your true labels
# Assuming pred is your predicted labels 
report = classification_report(y_test, pred)

print("Classification Report:\n", report)


In [ ]:
from sklearn.metrics import accuracy_score


accuracy = accuracy_score(y_test, pred)

print("Accuracy:", accuracy)


In [ ]:
import tkinter as tk
from PIL import Image, ImageTk

BACKGROUND_IMAGE_PATH = "C:/Users/aruna/Downloads/ai.png"
loaded_vectorizer=joblib.load('vectorizer.joblib')
# Create a function to load and resize images
def load_and_resize_image(image_path, size):
    image = Image.open(image_path)
    image = image.resize(size, Image.ANTIALIAS)
    return ImageTk.PhotoImage(image)

# Create the Tkinter window
app = tk.Tk()
app.title("AI Generation Detection")
app.geometry("600x400")  # Set window size
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()

# Load and resize images
background_image = load_and_resize_image(BACKGROUND_IMAGE_PATH, (screen_width,screen_height))

def predict_input():
    input_text = entry.get()
    
    # Vectorize the input text
    input_vectorized = loaded_vectorizer.transform([input_text])

    # Make a prediction
    prediction = ensemble.predict_proba(input_vectorized)

    if prediction[0, 1] > prediction[0, 0]:
        result_label.config(text="The input is AI generated")
    else:
        result_label.config(text=f"The input is not AI generated")

# Set window icon using PhotoImage
background_label = tk.Label(app)
background_label.place(relwidth=1, relheight=1)
background_label.image = background_image
background_label.configure(image=background_image)

label_font = ('Arial', 24)
entry_font = ('Arial', 18)
button_font = ('Arial', 18, 'bold')
result_font = ('Arial', 18, 'italic')

label = tk.Label(app, text="Enter a text:", font=label_font, bg="lightgreen")
label.pack(pady=10)

entry = tk.Entry(app, width=50, font=entry_font)  # Adjust height as needed
entry.pack(pady=10)

predict_button = tk.Button(app, text="Predict", command=predict_input, font=button_font, bg="green", fg="white")
predict_button.pack(pady=10)

result_label = tk.Label(app, text="Prediction result will be displayed here.", font=result_font, bg="white")
result_label.pack(pady=10)

# Start the GUI main loop
app.mainloop()
